In [4]:
lshell = [1,2,3,4,5,6]
hup = [
    {'SRAM_SEFI':1.22059E-12,'SRAM_SEL':1.04963E-16,'MCU_SEFI':5.13121E-08,'MCU_SEL':4.51608E-16,'LREG_SEGR':2.16558E-16},
    {'SRAM_SEFI':7.37521E-12,'SRAM_SEL':4.89213E-14,'MCU_SEFI':5.60427E-08,'MCU_SEL':2.80857E-13,'LREG_SEGR':1.22546E-13},
    {'SRAM_SEFI':1.72234E-11,'SRAM_SEL':1.05759E-13,'MCU_SEFI':8.32975E-10,'MCU_SEL':7.32052E-13,'LREG_SEGR':2.76040E-13},
    {'SRAM_SEFI':3.14337E-11,'SRAM_SEL':3.68747E-13,'MCU_SEFI':1.07664E-10,'MCU_SEL':2.21474E-12,'LREG_SEGR':9.42964E-13},
    {'SRAM_SEFI':4.35323E-11,'SRAM_SEL':8.84516E-13,'MCU_SEFI':1.41345E-10,'MCU_SEL':3.85914E-12,'LREG_SEGR':1.99045E-12},
    {'SRAM_SEFI':4.57481E-11,'SRAM_SEL':9.55653E-13,'MCU_SEFI':1.47896E-10,'MCU_SEL':4.12427E-12,'LREG_SEGR':2.14415E-12},
]
import pandas as pd
pd.options.plotting.backend = 'plotly'
hup = pd.DataFrame(hup,index=lshell)
hup.plot(log_y=True)

In [4]:
lshell = [1,2,3,4,5,6]
pup = [
    {'SRAM_SEFI':8.46710E-07,'MCU_SEFI':2.03155E-06},
    {'SRAM_SEFI':2.28263E-08,'MCU_SEFI':7.78840E-08},
    {'SRAM_SEFI':3.04213E-10,'MCU_SEFI':6.08426E-10},
    {'SRAM_SEFI':4.03984E-10,'MCU_SEFI':8.07967E-10},
    {'SRAM_SEFI':4.36917E-10,'MCU_SEFI':8.74082E-10},
    {'SRAM_SEFI':4.42122E-10,'MCU_SEFI':8.86007E-10},
]
import pandas as pd
pd.options.plotting.backend = 'plotly'
pup = pd.DataFrame(pup,index=lshell)
pup.plot(log_y=True)

In [5]:
lshell = [1,2,3,4,5,6]
dose = [
    {'rad/s':2.170E-01},
    {'rad/s':1.791E-01},
    {'rad/s':1.860E-03},
    {'rad/s':3.474E-07},
    {'rad/s':3.974E-07},
    {'rad/s':4.129E-07},
]
import pandas as pd
pd.options.plotting.backend = 'plotly'
dose = pd.DataFrame(dose,index=lshell)
dose.plot(log_y=True)

In [15]:
import pandas as pd
import numpy as np

creme_files = 'trajectory/CREME96'
lshells = [1.0,1.2,1.5,1.8,2.2,2.7,3.3,4.0,4.9]

def read_creme(file):
    file = open(file).read().split('\n')
    file_id = file.pop(0)[-5:]
    if file_id.endswith(' 4'): headers = [f'Z={i}' for i in range(1,93)]
    if file_id.endswith(' 5'): headers = ['LET (MeV-cm2/g)']
    if file_id.endswith('10'): 
    table, new, i = pd.DataFrame(), [], 0
    for line in file[1:]:
        if line.startswith(' %'): continue
        if line.endswith(file_id):
            x = np.geomspace(float(line.split('  ')[0]),float(line.split('  ')[1]),int(line.split('   ')[1]))
            continue
        if line == '': 
            if len(new) > 0: table[headers[len(table.columns)]] = pd.Series(new,index=x).astype(float)
            new = []
            continue
        new += line[1:].split('  ')
    if len(new) > 0: table[headers[len(table.columns)]] = pd.Series(new,index=x).astype(float)
    return table
        

def read(creme='trajectory/CREME96',prefix='gto',lshells=[1.0,1.19,1.41,1.68,1.99,2.37,2.82,3.35,3.98,4.73]):
    if creme == None: return -1
    dose_rate = pd.DataFrame(columns=lshells,index=['Dose Rate (rad/s)'])
    gcr_spec,let_spec = [],pd.DataFrame(columns=lshells)
    for i,l in enumerate(lshells):
        dose_rate[l] = [float(open(f'{creme}/{prefix}{i+1}.dse').read().split('\n')[2].split('=')[1].split('rad')[0])]
        gcr_spec.append(read_creme(f'{creme}/{prefix}{i+1}.tfx'))
        let_spec[l] = read_creme(f'{creme}/{prefix}{i+1}.let')
    dose_rate.columns.name = 'L-shell'
    let_spec.index.name = 'LET (MeV-cm2/g)'
    let_spec.columns.name = 'L-shell'
    return dose_rate,let_spec,gcr_spec

0.01297

In [3]:
from scipy.interpolate import interp1d
D = interp1d(list(dose_rate.columns),list(np.log(dose_rate.iloc[0])),kind='cubic')

import plotly.express as px
l = np.linspace(1,max(dose_rate.columns),1000)
px.line(x=l,y=np.exp(D(l)),log_y=True).add_traces(list(px.bar(dose_rate.T).select_traces()))

In [12]:
print(*np.around(np.geomspace(1,5.62,10),2),sep=',')

1.0,1.21,1.47,1.78,2.15,2.61,3.16,3.83,4.64,5.62


In [93]:
import pandas as pd
rates = pd.DataFrame({
    'D':1.297E-05, # krad/s
    'SRAM_SEL':1.19749E-02 + 1.24587E-10,
    'MCU_SEFI':9.73404E-05 + 2.62931E-11,
    'MCU_SEL':1.19749E-02 + 4.43484E-12,
    'LREG_SEGR':4.37546E-08
},index=pd.Series([0],name='L'))
rates

,D,SRAM_SEL,MCU_SEFI,MCU_SEL,LREG_SEGR
L,,,,,
0,0.000013,0.011975,0.000097,0.011975,4.375460e-08


In [ ]:
doses = {
    'SRAM_TID':{'mu':5,'sigma':0.1},
    'MCU_TID':{'mu':2,'sigma':0.1},
    'LREG_TID':{'mu':2,'sigma':0.1}
}

In [99]:
import numpy as np
tstep = 1 * 24 * 3600
t = np.linspace(0,tstep,10000)
P = pd.concat([rates] * len(t))
P.index = t
P = P.mul(t,axis=0)
P[P.columns[1:]] = 1 - np.exp(-P[P.columns[1:]])
P

,D,SRAM_SEL,MCU_SEFI,MCU_SEL,LREG_SEGR
0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
8.640864,0.000112,0.098300,0.000841,0.098300,3.780775e-07
17.281728,0.000224,0.186937,0.001681,0.186937,7.561548e-07
25.922592,0.000336,0.266861,0.002520,0.266861,1.134232e-06
34.563456,0.000448,0.338929,0.003359,0.338929,1.512309e-06
...,...,...,...,...,...
86365.436544,1.120160,1.000000,0.999777,1.000000,3.771754e-03
86374.077408,1.120272,1.000000,0.999777,1.000000,3.772131e-03
86382.718272,1.120384,1.000000,0.999777,1.000000,3.772507e-03
86391.359136,1.120496,1.000000,0.999777,1.000000,3.772884e-03


In [100]:
from radstats import models
P['SRAM_TID'] = models.lognormal(P['D'],5,0.1)
P['MCU_TID'] = models.lognormal(P['D'],2,0.1)
P['LREG_TID'] = models.lognormal(P['D'],2,0.1)
P['WDOG_TID'] = models.lognormal(P['D'],4,0.1)
P

/Users/stephen/Desktop/rad-fault-tree/thesis/radstats/models.py:13: RuntimeWarning:

divide by zero encountered in log



,D,SRAM_SEL,MCU_SEFI,MCU_SEL,LREG_SEGR,SRAM_TID,MCU_TID,LREG_TID,WDOG_TID
0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.0,0.0,0.0
8.640864,0.000112,0.098300,0.000841,0.098300,3.780775e-07,0.0,0.0,0.0,0.0
17.281728,0.000224,0.186937,0.001681,0.186937,7.561548e-07,0.0,0.0,0.0,0.0
25.922592,0.000336,0.266861,0.002520,0.266861,1.134232e-06,0.0,0.0,0.0,0.0
34.563456,0.000448,0.338929,0.003359,0.338929,1.512309e-06,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
86365.436544,1.120160,1.000000,0.999777,1.000000,3.771754e-03,0.0,0.0,0.0,0.0
86374.077408,1.120272,1.000000,0.999777,1.000000,3.772131e-03,0.0,0.0,0.0,0.0
86382.718272,1.120384,1.000000,0.999777,1.000000,3.772507e-03,0.0,0.0,0.0,0.0
86391.359136,1.120496,1.000000,0.999777,1.000000,3.772884e-03,0.0,0.0,0.0,0.0


In [91]:
import plotly.express as px
px.line(P)